In [1]:
import sys, subprocess
def ensure(pkgs): subprocess.check_call([sys.executable, "-m", "pip", "install", "--quiet", *pkgs])
ensure(["albumentations", "segmentation_models_pytorch", "tqdm", "torchmetrics>=1.3.0", "jupyter", "ipywidgets"])

## Датасетик качаем

| Критерий                             | Обоснование                                                                                                                                                                                                                        |
| ------------------------------------ | ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **Реальная практическая значимость** | Ранняя и точная сегментация границ кожных поражений (меланома, невусы, базальноклеточный рак) критична для поддержки решений дерматологов и систем компьютерного зрения, снижает риск пропустить злокачественные изменения.        |
| **Разнообразие и баланс классов**    | ≈2594 изображений с масками «поражение/фон» разного цвета кожи, типа и локализации участков; одноклассовая (бинарная) задача, но большая вариативность размеров, формы, текстуры и освещения.                                      |
| **Размер и доступность**             | Общий объём ≈600 МБ (JPEG+PNG), легко обрабатывается на студенческом ПК или в Colab; подходит для глубокого обучения без распределённых систем.                                                                                    |
| **Качество аннотаций**               | Бинарные маски созданы и верифицированы экспертами-дерматологами из ISIC Archive; чёткие границы способствуют обучению моделей с высокой точностью сегментации.                                                                    |

In [1]:
!pip install kaggle --quiet

In [2]:
!kaggle datasets download -d tschandl/isic2018-challenge-task1-data-segmentation --unzip -p ./data

In [6]:
!unzip archive.zip ./data

/bin/sh: 1: unzip: not found


In [2]:
from pathlib import Path

root = Path("/home/data")

img_dir  = root / "ISIC2018_Task1-2_Training_Input"
mask_dir = root / "ISIC2018_Task1_Training_GroundTruth"

## Импорты

In [3]:
import datetime
import math
import platform
import random
import warnings
import functools
import glob
import time

import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import numpy as np
import pandas as pd
import segmentation_models_pytorch as smp
import torch
from torch import nn, optim
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import DataLoader, Dataset, random_split
from torchmetrics.classification import BinaryF1Score as Dice
from torchmetrics.classification import BinaryJaccardIndex as JaccardIndex
from torchmetrics.classification import BinaryAccuracy as Accuracy
from tqdm.auto import tqdm

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

torch.backends.cudnn.benchmark = True

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

IMG_SIZE = 320

## Полезные штучки

### Датасет + аугментации

| Этап                             | Операции                                                                                                                                                                                                                                                                                     |
| -------------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **Аугментации для обучения**     | - `HorizontalFlip` + `VerticalFlip` (случайные отражения)<br>- `ElasticTransform` (α=120, σ=15, p=0.3)<br>- `ColorJitter` (±20 % яркости/контраста/насыщенности, ±10 % оттенка)<br>- `Resize` → `IMG_SIZE×IMG_SIZE`<br>- `Normalize` + `ToTensorV2` (пиксели \[0,1], транспонирование маски) |
| **Преобразования для валидации** | - `Resize` → `IMG_SIZE×IMG_SIZE`<br>- `Normalize` + `ToTensorV2` (без случайных операций)                                                                                                                                                                                                    |
| **Разделение 80 / 20**           | - 80 % (`train_ds`) с `RAND_AUG`, `shuffle=True`<br>- 20 % (`val_ds`) с `BASE_AUG`, `shuffle=False`<br>- `batch_size` задаётся в `make_loaders()`                                                                                                                                            |

In [4]:
@functools.lru_cache(maxsize=None)
def read_rgb(path):
    return cv2.imread(path)[:, :, ::-1].copy()

class ISICDataset(Dataset):
    def __init__(
        self,
        img_dir,
        mask_dir,
        augment,
    ):
        self.imgs = sorted(
            glob.glob(str(img_dir / "*.jpg"))
        )
        self.masks = sorted(
            glob.glob(str(mask_dir / "*.png"))
        )
        self.aug = augment

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, index):
        img = read_rgb(self.imgs[index])
        mask = (
            cv2.imread(self.masks[index], 0) > 0
        ).astype("float32")[..., None]
        out = self.aug(image=img, mask=mask)
        img, mask = out["image"], out["mask"]
        #img = torch.from_numpy(img.transpose(2, 0, 1) / 255.0).float()
        #mask = torch.from_numpy(mask.transpose(2, 0, 1)).float()
        return img, mask

BASE_AUG = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.Normalize(mean=(0.0, 0.0, 0.0), std=(1.0, 1.0, 1.0)),
    ToTensorV2(transpose_mask=True),
])

RAND_AUG = A.Compose([
    A.HorizontalFlip(), A.VerticalFlip(),
    A.ElasticTransform(alpha=120, sigma=15, p=0.3),
    A.ColorJitter(0.2,0.2,0.2,0.1),
    A.Normalize(mean=(0.0, 0.0, 0.0), std=(1.0, 1.0, 1.0)),
    ToTensorV2(transpose_mask=True),
])

full_ds = ISICDataset(
    img_dir,
    mask_dir,
    BASE_AUG,
)
train_len = int(0.8 * len(full_ds))
val_len = len(full_ds) - train_len
train_ds, val_ds = random_split(
    full_ds,
    [train_len, val_len],
)


def make_loaders(
    batch: int,
    augment: bool,
):
    train_ds.dataset.aug = (
        RAND_AUG if augment else BASE_AUG
    )
    val_ds.dataset.aug = BASE_AUG
    return (
        DataLoader(
            train_ds,
            batch_size=batch,
            shuffle=True,
            num_workers=2,
            pin_memory=True,
        ),
        DataLoader(
            val_ds,
            batch_size=batch,
            shuffle=False,
            num_workers=2,
            pin_memory=True,
        ),
    )

### Метрики

- **Dice (коэффициент Диcа)**  
  - Оценивает степень перекрытия предсказанной области \(P\) и эталонной маски \(G\), с дополнительным весом на пересечение.  
  - Хорошо работает при сильном дисбалансе классов (малые поражения).  

- **IoU (Jaccard Index)**   
  - Измеряет отношение пересечения к объединению областей; более строгая метрика, чем Dice (каждый невключённый пиксель сильнее штрафуется).  

- **Accuracy (точность)**  
  - Доля правильно классифицированных пикселей (True Positives + True Negatives от общего числа).  
  - Может давать завышенные результаты, если фон (Negative класс) сильно преобладает.  

- **Практические детали**  
  - По умолчанию все метрики порогируют вероятности на 0.5.  
  - В начале каждой оценки вызывается `reset_metrics()` для обнуления накопленных значений.  
  - После прохода по всем батчам `compute()` возвращает усреднённый результат за весь загрузчик.  

In [5]:
dice_metric = Dice().to(DEVICE)
iou_metric  = JaccardIndex().to(DEVICE)
acc_metric  = Accuracy().to(DEVICE)

def reset_metrics():
    dice_metric.reset()
    iou_metric.reset()
    acc_metric.reset()

@torch.no_grad()
def evaluate(model, loader):
    reset_metrics()
    model.eval()

    for images, masks in loader:
        images = images.to(DEVICE)
        masks = masks.to(DEVICE)

        predictions = torch.sigmoid(model(images))
        dice_metric.update(predictions, masks)
        iou_metric.update(predictions, masks)
        acc_metric.update(predictions, masks)

    dice_score = float(dice_metric.compute())
    iou_score = float(iou_metric.compute())
    accuracy_score = float(acc_metric.compute())

    return dice_score, iou_score, accuracy_score

### Функции обучения

In [6]:
from tqdm.auto import tqdm
from torch.cuda.amp import autocast, GradScaler

def train_model(
    model: nn.Module,
    cfg: dict,
) -> tuple[nn.Module, tuple[float, float, float]]:
    train_dl, val_dl = make_loaders(
        batch   = cfg["batch"],
        augment = cfg["augment"],
    )

    model = model.to(DEVICE)
    # model = torch.compile(model)

    scaler     = GradScaler()
    optimizer  = optim.AdamW(model.parameters(),
                             lr=cfg["lr"], weight_decay=1e-4)
    sched      = optim.lr_scheduler.OneCycleLR(
        optimizer, max_lr=cfg["lr"],
        epochs=cfg["epochs"],
        steps_per_epoch=len(train_dl)//cfg["acc_steps"],
        pct_start=0.1, div_factor=25, final_div_factor=100,
    )

    loss_fn            = cfg["loss"]
    accumulation_steps = cfg["acc_steps"]

    for epoch in range(1, cfg["epochs"] + 1):
        model.train()
        grad_step   = 0
        running_loss= 0.0
        start_time  = time.time()

        for images, masks in tqdm(
            train_dl,
            desc=f"Epoch {epoch}/{cfg['epochs']}",
            leave=False,
        ):
            images, masks = images.to(DEVICE), masks.to(DEVICE)

            if grad_step == 0:
                optimizer.zero_grad(set_to_none=True)

            with autocast():
                preds = model(images)
                loss  = loss_fn(preds, masks) / accumulation_steps

            scaler.scale(loss).backward()
            running_loss += loss.item() * accumulation_steps
            grad_step += 1

            if grad_step == accumulation_steps:
                scaler.step(optimizer)
                scaler.update()
                sched.step()
                grad_step = 0

        dice, iou, acc = evaluate(model, val_dl)
        epoch_time     = time.time() - start_time
        avg_loss       = running_loss / len(train_dl)

        print(
            f"Ep {epoch:02d}/{cfg['epochs']} "
            f"| loss {avg_loss:.4f} "
            f"| Dice {dice:.4f} "
            f"| IoU {iou:.4f} "
            f"| Acc {acc:.4f} "
            f"| {epoch_time:.1f}s/ep"
        )

    return model, (dice, iou, acc)

### Лосс

- **BCEWithLogitsLoss**  
  - Реализует бинарную кросс-энтропию с учётом логитов (внутри применяется `sigmoid`).  
  - Штрафует разницу по-пиксельно, особенно чувствителен к редким ошибкам при большом фоне.  

- **Combo Loss**  
  - Балансирует пиксельную точность (BCE) и глобальное перекрытие (Dice).  
  - Сглаживание «+1» в Dice предотвращает деление на ноль, когда маска пустая.

In [7]:
bce_loss = nn.BCEWithLogitsLoss()

def combo_loss(
    predictions: torch.Tensor,
    targets: torch.Tensor,
) -> torch.Tensor:
    dice_numerator = (
        2 * (predictions.sigmoid() * targets).sum() + 1
    )
    dice_denominator = (
        predictions.sigmoid().sum() + targets.sum() + 1
    )
    dice_score = dice_numerator / dice_denominator

    loss = (
        0.5 * bce_loss(predictions, targets)
        + 0.5 * (1 - dice_score)
    )

    return loss

## Свои модели

### Просто моделька

| Блок        | Что делает                                                                                                                                                            |
| :---------- | :-------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **e1**      | Два подряд 3 × 3 Conv (in=3 → out=f) + ReLU; сохраняет пространственный размер, извлекает начальные признаки из RGB-изображения.                                      |
| **e2**      | MaxPool2d 2 × 2 (H,W → H/2,W/2) + 3 × 3 Conv (in=f → out=2f) + ReLU; уменьшает разрешение вдвое, удваивает число каналов для более глубоких признаков.                |
| **e3**      | MaxPool2d 2 × 2 + 3 × 3 Conv (in=2f → out=4f) + ReLU; повторяет снижение разрешения и расширение каналов для вычленения ещё более абстрактных признаков.              |
| **mid**     | MaxPool2d 2 × 2 + 3 × 3 Conv (in=4f → out=8f) + ReLU; «бутылочное горлышко» U-Net — максимальная глубина, здесь сосредоточены самые глобальные признаки объекта.      |
| **u2 + d2** | `ConvTranspose2d(in=8f→4f, k=2, s=2)` ↑2 → конкатенация с e3 (4f+4f=8f) → 3 × 3 Conv (in=8f → out=4f) + ReLU; первый этап декодера, восстанавливает разрешение вдвое. |
| **u1 + d1** | `ConvTranspose2d(in=4f→2f, k=2, s=2)` ↑2 → конкатенация с e2 (2f+2f=4f) → 3 × 3 Conv (in=4f → out=2f) + ReLU; второй уровень декодера.                                |
| **u0 + d0** | `ConvTranspose2d(in=2f→f, k=2, s=2)` ↑2 → конкатенация с e1 (f+f=2f) → 3 × 3 Conv (in=2f → out=f) + ReLU; финальный уровень декодера, возвращает исходное разрешение. |
| **head**    | 1 × 1 Conv (in=f → out=1); проекция f-канального тензора в одноканальную логит-маску для бинарной сегментации.                                                        |
- Узел «конкатенации» обеспечивает передачу локальных деталей из энкодера в декодер.
- ConvTranspose2d разворачивает пространственные размеры (upsampling), вместо интерполяции.
- ReLU после каждого свёрточного слоя позволяет сети моделировать нелинейные отношения в данных.

In [8]:
class TinyUNet(nn.Module):
    def __init__(self, f: int = 32):
        super().__init__()
        self.e1 = nn.Sequential(
            nn.Conv2d(
                in_channels=3,
                out_channels=f,
                kernel_size=3,
                stride=1,
                padding=1,
            ),
            nn.ReLU(),
            nn.Conv2d(
                in_channels=f,
                out_channels=f,
                kernel_size=3,
                stride=1,
                padding=1,
            ),
            nn.ReLU(),
        )

        self.e2 = nn.Sequential(
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(
                in_channels=f,
                out_channels=2 * f,
                kernel_size=3,
                stride=1,
                padding=1,
            ),
            nn.ReLU(),
        )

        self.e3 = nn.Sequential(
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(
                in_channels=2 * f,
                out_channels=4 * f,
                kernel_size=3,
                stride=1,
                padding=1,
            ),
            nn.ReLU(),
        )

        self.mid = nn.Sequential(
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(
                in_channels=4 * f,
                out_channels=8 * f,
                kernel_size=3,
                stride=1,
                padding=1,
            ),
            nn.ReLU(),
        )

        self.u2 = nn.ConvTranspose2d(
            in_channels=8 * f,
            out_channels=4 * f,
            kernel_size=2,
            stride=2,
        )
        self.d2 = nn.Conv2d(
            in_channels=8 * f,
            out_channels=4 * f,
            kernel_size=3,
            stride=1,
            padding=1,
        )

        self.u1 = nn.ConvTranspose2d(
            in_channels=4 * f,
            out_channels=2 * f,
            kernel_size=2,
            stride=2,
        )
        self.d1 = nn.Conv2d(
            in_channels=4 * f,
            out_channels=2 * f,
            kernel_size=3,
            stride=1,
            padding=1,
        )

        self.u0 = nn.ConvTranspose2d(
            in_channels=2 * f,
            out_channels=f,
            kernel_size=2,
            stride=2,
        )
        self.d0 = nn.Conv2d(
            in_channels=2 * f,
            out_channels=f,
            kernel_size=3,
            stride=1,
            padding=1,
        )

        self.head = nn.Conv2d(
            in_channels=f,
            out_channels=1,
            kernel_size=1,
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        e1 = self.e1(x)
        e2 = self.e2(e1)
        e3 = self.e3(e2)
        m = self.mid(e3)

        d2 = self.u2(m)
        d2 = torch.cat([d2, e3], dim=1)
        d2 = torch.relu(self.d2(d2))

        d1 = self.u1(d2)
        d1 = torch.cat([d1, e2], dim=1)
        d1 = torch.relu(self.d1(d1))

        d0 = self.u0(d1)
        d0 = torch.cat([d0, e1], dim=1)
        d0 = torch.relu(self.d0(d0))

        return self.head(d0)

### Улучшенная типо

| Блок                               | Что делает                                                                                                                                            |
| :--------------------------------- | :---------------------------------------------------------------------------------------------------------------------------------------------------- |
| **patch\_conv**                    | 2D-свёртка патчами (kernel=patch\_size, stride=patch\_size): из RGB-изображения формирует тензор размером (B, dim, H/patch, W/patch).                 |
| **cls\_token + pos\_embed**        | Добавляет learnable токен класса и позиционные эмбеддинги к последовательности патч-векторов (размер N\_patches+1).                                   |
| **encoder**                        | TransformerEncoder из `depth` слоёв с `nhead` головами и FFN размером `dim*mlp`; обрабатывает токены, моделируя глобальные взаимосвязи между патчами. |
| **reshape → feature map**          | Убирает первый (cls) токен, транспонирует и ресайзит последовательность обратно в карту признаков (B, dim, H/patch, W/patch).                         |
| **up (ConvTranspose2d + Conv1×1)** | Расширяет карту признаков вдвое (dim→dim/2), применяет ReLU и 1×1-свёртку для получения одноканального предсказания низкого разрешения.               |
| **interpolate**                    | Билинейно масштабирует предсказание с (H/patch, W/patch) обратно до оригинального (H, W) без артефактов.                                              |

- Патчи позволяют Transformer-модулям работать на более низком разрешении, экономя память.
- Positional embedding даёт информацию о расположении патча в картинке.
- Использование ConvTranspose2d вместо простой интерполяции добавляет обучаемые параметры при восстановлении пространственных деталей.
- Билинейная интерполяция на конце гарантирует плавное увеличение до исходного размера без пикселизации.

In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PatchViTSeg(nn.Module):
    def __init__(
        self,
        img: int = IMG_SIZE,
        patch: int = 16,
        dim: int = 256,
        depth: int = 4,
        heads: int = 4,
        mlp: int = 2,
    ):
        super().__init__()
        self.patch_size = patch

        self.patch_conv = nn.Conv2d(
            in_channels=3,
            out_channels=dim,
            kernel_size=self.patch_size,
            stride=self.patch_size,
        )
        num_patches = (img // self.patch_size) ** 2

        self.cls_token = nn.Parameter(torch.zeros(1, 1, dim))
        self.pos_embed = nn.Parameter(
            torch.randn(1, num_patches + 1, dim) * 0.02
        )

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=dim,
            nhead=heads,
            dim_feedforward=dim * mlp,
            batch_first=True,
            norm_first=True,
        )
        self.encoder = nn.TransformerEncoder(
            encoder_layer,
            num_layers=depth,
        )

        self.up = nn.Sequential(
            nn.ConvTranspose2d(
                in_channels=dim,
                out_channels=dim // 2,
                kernel_size=2,
                stride=2,
            ),
            nn.ReLU(),
            nn.Conv2d(
                in_channels=dim // 2,
                out_channels=1,
                kernel_size=1,
            ),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        batch_size = x.size(0)

        patches = self.patch_conv(x)
        patches = patches.flatten(2).transpose(1, 2)

        tokens = torch.cat(
            [
                self.cls_token.expand(batch_size, -1, -1),
                patches,
            ],
            dim=1,
        ) + self.pos_embed

        encoded = self.encoder(tokens)
        encoded = encoded[:, 1:]
        encoded = encoded.transpose(1, 2).reshape(
            batch_size,
            -1,
            IMG_SIZE // self.patch_size,
            IMG_SIZE // self.patch_size,
        )

        preds_small = self.up(encoded)

        preds_full = F.interpolate(
            preds_small,
            size=(IMG_SIZE, IMG_SIZE),
            mode="bilinear",
            align_corners=False
        )
        return preds_full

## Конфиги

| Модель            | Бэкенд    | Аугментации    | Лосс          | batch | epochs | lr      | acc\_steps |
| ----------------- | --------- | -------------- | ------------- | ----- | ------ | ------- | ---------- |
| **unet\_base**    | ResNet-34 | Нет            | BCEWithLogits | 20    | 3      | 4.2 e-3 | 1          |
| **deeplab\_base** | MiT-B2    | Нет            | BCEWithLogits | 9     | 3      | 4.2 e-3 | 1          |
| **unet\_plus**    | ResNet-34 | Да (RAND\_AUG) | ComboLoss     | 14    | 4      | 1.4 e-3 | 1          |
| **deeplab\_plus** | MiT-B2    | Да (RAND\_AUG) | ComboLoss     | 7     | 4      | 1.4 e-3 | 1          |

In [10]:
CFG = {
    "unet_base": dict(
        batch=20, epochs=3,  lr=4.2e-3,
        augment=False, loss=bce_loss, acc_steps=1,
        create=lambda: smp.Unet("resnet34", encoder_weights="imagenet",
                                classes=1, activation=None)
    ),

    "deeplab_base": dict(
        batch=9,  epochs=3,  lr=4.2e-3,
        augment=False, loss=bce_loss, acc_steps=1,
        create=lambda: smp.DeepLabV3Plus("mit_b2", encoder_weights="imagenet",
                                         classes=1, activation=None)
    ),

    "unet_plus": dict(
        batch=14, epochs=4,  lr=1.4e-3,
        augment=True,  loss=combo_loss, acc_steps=1,
        create=lambda: smp.Unet("resnet34", encoder_weights="imagenet",
                                classes=1, activation=None)
    ),

    "deeplab_plus": dict(
        batch=7,  epochs=4,  lr=1.4e-3,
        augment=True,  loss=combo_loss, acc_steps=1,
        create=lambda: smp.DeepLabV3Plus("mit_b2", encoder_weights="imagenet",
                                         classes=1, activation=None)
    ),

    "tiny_unet": dict(
        batch=32, epochs=5,  lr=1.4e-3,
        augment=True,  loss=combo_loss, acc_steps=1, create=TinyUNet
    ),

    "vit_seg": dict(
        batch=4,  epochs=5,  lr=7e-4,
        augment=True,  loss=combo_loss, acc_steps=2, create=PatchViTSeg
    ),
}

In [ ]:
results = []

for name, cfg in CFG.items():
    print(f"\n=== {name} ===")

    model = cfg["create"]()

    _, metrics = train_model(
        model=model,
        cfg=cfg,
    )

    results.append(
        [
            name,
            *metrics,
        ]
    )


=== unet_base ===


Epoch 1/3:   0%|          | 0/104 [00:00<?, ?it/s]

Ep 01/3 | loss 0.2733 | Dice 0.5910 | IoU 0.4194 | Acc 0.7275 | 342.8s/ep


Epoch 2/3:   0%|          | 0/104 [00:00<?, ?it/s]

Ep 02/3 | loss 0.1871 | Dice 0.8016 | IoU 0.6688 | Acc 0.9227 | 345.8s/ep


...

Epoch 1/5:   0%|          | 0/65 [00:00<?, ?it/s]

Ep 01/5 | loss 0.5529 | Dice 0.5853 | IoU 0.4137 | Acc 0.8464 | 362.8s/ep


Epoch 2/5:   0%|          | 0/65 [00:00<?, ?it/s]

Ep 02/5 | loss 0.4242 | Dice 0.6079 | IoU 0.4367 | Acc 0.8604 | 373.3s/ep


Epoch 3/5:   0%|          | 0/65 [00:00<?, ?it/s]

In [14]:
CFG = {
    "tiny_unet": dict(
        batch=32, epochs=5,  lr=1.4e-3,
        augment=True,  loss=combo_loss, acc_steps=1, create=TinyUNet
    ),

    "vit_seg": dict(
        batch=4,  epochs=5,  lr=7e-4,
        augment=True,  loss=combo_loss, acc_steps=2, create=PatchViTSeg
    ),
}

In [12]:
results = []

for name, cfg in CFG.items():
    print(f"\n=== {name} ===")

    model = cfg["create"]()

    _, metrics = train_model(
        model=model,
        cfg=cfg,
    )

    results.append(
        [
            name,
            *metrics,
        ]
    )


=== tiny_unet ===


Epoch 1/5:   0%|          | 0/65 [00:00<?, ?it/s]

Ep 01/5 | loss 0.5691 | Dice 0.5437 | IoU 0.3733 | Acc 0.8438 | 374.4s/ep


Epoch 2/5:   0%|          | 0/65 [00:00<?, ?it/s]

Ep 02/5 | loss 0.4302 | Dice 0.6101 | IoU 0.4389 | Acc 0.8611 | 357.7s/ep


...

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f9779cdb9a0>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1462, in _shutdown_workers
    if w.is_alive():
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f9779cdb9a0>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1462, in _shutdown_workers
    if w.is_alive():
  File "/op

Ep 05/5 | loss 0.3171 | Dice 0.7176 | IoU 0.5595 | Acc 0.8859 | 407.4s/ep

=== vit_seg ===


/opt/conda/lib/python3.10/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/5:   0%|          | 0/519 [00:00<?, ?it/s]

NameError: name 'patch' is not defined

In [21]:
CFG = {
    "vit_seg": dict(
        batch=4,  epochs=5,  lr=7e-4,
        augment=True,  loss=combo_loss, acc_steps=2, create=PatchViTSeg
    ),
}

In [22]:
results = []

for name, cfg in CFG.items():
    print(f"\n=== {name} ===")

    model = cfg["create"]()

    _, metrics = train_model(
        model=model,
        cfg=cfg,
    )

    results.append(
        [
            name,
            *metrics,
        ]
    )


=== vit_seg ===


Epoch 1/5:   0%|          | 0/519 [00:00<?, ?it/s]

Ep 01/5 | loss 0.3919 | Dice 0.7910 | IoU 0.6543 | Acc 0.9110 | 393.7s/ep


Epoch 2/5:   0%|          | 0/519 [00:00<?, ?it/s]

Ep 02/5 | loss 0.2683 | Dice 0.7713 | IoU 0.6277 | Acc 0.9125 | 375.9s/ep


...

Ep 03/5 | loss 0.2405 | Dice 0.7848 | IoU 0.6458 | Acc 0.9165 | 358.6s/ep


Epoch 4/5:   0%|          | 0/519 [00:00<?, ?it/s]

Ep 04/5 | loss 0.2024 | Dice 0.8277 | IoU 0.7061 | Acc 0.9289 | 371.8s/ep


Epoch 5/5:   0%|          | 0/519 [00:00<?, ?it/s]

Ep 05/5 | loss 0.1884 | Dice 0.8391 | IoU 0.7228 | Acc 0.9316 | 371.1s/ep


## Результаты

In [ ]:
df = pd.DataFrame(results, columns=["Model","Dice","IoU","PixAcc"])
print("\n===== Итоговые метрики (GPU:", torch.cuda.get_device_name(0), ") =====")
print(df.to_string(index=False))

| Модель           | Эпохи |   Loss   |  Dice   |   IoU   |   Acc   | Время (с/эп) |
|------------------|:-----:|:--------:|:-------:|:-------:|:-------:|:------------:|
| **unet_base**    |  3/3  |  0.1466  | 0.8607  | 0.7555  | 0.9410  |    342.3     |
| **deeplab_base** |  3/3  |  0.2126  | 0.8060  | 0.6750  | 0.9229  |    379.9     |
| **unet_plus**    |  4/4  |  0.1219  | 0.8879  | 0.7983  | 0.9515  |    351.2     |
| **deeplab_plus** |  4/4  |  0.2072  | 0.8339  | 0.7151  | 0.9321  |    363.3     |
| **tiny_unet**    |  5/5  |  0.3171  | 0.7176  | 0.5595  | 0.8859  |    407.4     |
| **vit_seg**      |  5/5  |  0.1884  | 0.8391  | 0.7228  | 0.9316  |    371.1     |